In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
import requests
import json

pt_df = pd.read_csv("./data/pt_ratings.csv")
salaries_df = pd.read_csv("./data/salaries.csv")
rmp_df = pd.read_csv("./data/rmp_ratings.csv")


In [5]:
def rmp_search(name):
  query = """query NewSearchTeachersQuery(
          $query: TeacherSearchQuery!
      ) {
      newSearch {
          teachers(query: $query) {
          didFallback
          edges {
              cursor
              node {
              id
              legacyId
              firstName
              lastName
              school {
                  name
                  id
              }
              department
              }
          }
          }
      }
  }"""   

  variables = {"query": {"text": name}}
  url = "https://www.ratemyprofessors.com/graphql"
  basic = requests.auth.HTTPBasicAuth('test', 'test')
  res = requests.post(url, json={"query": query, "variables": variables}, auth=basic)

  data = json.loads(res.text)
  profId = ""

  try:
    profId = data['data']['newSearch']['teachers']['edges'][0]['node']['id']
  except:
    profId = "NA"

  return profId

def rmp_get_ratings(name):
  query = """query RatingsListQuery(
    $count: Int!
    $id: ID!
    $courseFilter: String
    $cursor: String
  ) {
    node(id: $id) {
      __typename
      ... on Teacher {
        ...RatingsList_teacher_4pguUW
      }
      id
    }
  }

  fragment RatingsList_teacher_4pguUW on Teacher {
    id
    legacyId
    lastName
    numRatings
    school {
      id
      legacyId
      name
      city
      state
      avgRating
      numRatings
    }
    ...Rating_teacher
    ...NoRatingsArea_teacher
    ratings(first: $count, after: $cursor, courseFilter: $courseFilter) {
      edges {
        cursor
        node {
          ...Rating_rating
          id
          __typename
        }
      }
      pageInfo {
        hasNextPage
        endCursor
      }
    }
  }

  fragment Rating_teacher on Teacher {
    ...RatingFooter_teacher
    ...RatingSuperHeader_teacher
    ...ProfessorNoteSection_teacher
  }

  fragment NoRatingsArea_teacher on Teacher {
    lastName
    ...RateTeacherLink_teacher
  }

  fragment Rating_rating on Rating {
    comment
    flagStatus
    createdByUser
    teacherNote {
      id
    }
    ...RatingHeader_rating
    ...RatingSuperHeader_rating
    ...RatingValues_rating
    ...CourseMeta_rating
    ...RatingTags_rating
    ...RatingFooter_rating
    ...ProfessorNoteSection_rating
  }

  fragment RatingHeader_rating on Rating {
    date
    class
    helpfulRating
    clarityRating
    isForOnlineClass
  }

  fragment RatingSuperHeader_rating on Rating {
    legacyId
  }

  fragment RatingValues_rating on Rating {
    helpfulRating
    clarityRating
    difficultyRating
  }

  fragment CourseMeta_rating on Rating {
    attendanceMandatory
    wouldTakeAgain
    grade
    textbookUse
    isForOnlineClass
    isForCredit
  }

  fragment RatingTags_rating on Rating {
    ratingTags
  }

  fragment RatingFooter_rating on Rating {
    id
    comment
    adminReviewedAt
    flagStatus
    legacyId
    thumbsUpTotal
    thumbsDownTotal
    thumbs {
      userId
      thumbsUp
      thumbsDown
      id
    }
    teacherNote {
      id
    }
  }

  fragment ProfessorNoteSection_rating on Rating {
    teacherNote {
      ...ProfessorNote_note
      id
    }
    ...ProfessorNoteEditor_rating
  }

  fragment ProfessorNote_note on TeacherNotes {
    comment
    ...ProfessorNoteHeader_note
    ...ProfessorNoteFooter_note
  }

  fragment ProfessorNoteEditor_rating on Rating {
    id
    legacyId
    class
    teacherNote {
      id
      teacherId
      comment
    }
  }

  fragment ProfessorNoteHeader_note on TeacherNotes {
    createdAt
    updatedAt
  }

  fragment ProfessorNoteFooter_note on TeacherNotes {
    legacyId
    flagStatus
  }

  fragment RateTeacherLink_teacher on Teacher {
    legacyId
    numRatings
    lockStatus
  }

  fragment RatingFooter_teacher on Teacher {
    id
    legacyId
    lockStatus
    isProfCurrentUser
  }

  fragment RatingSuperHeader_teacher on Teacher {
    firstName
    lastName
    legacyId
    school {
      name
      id
    }
  }

  fragment ProfessorNoteSection_teacher on Teacher {
    ...ProfessorNote_teacher
    ...ProfessorNoteEditor_teacher
  }

  fragment ProfessorNote_teacher on Teacher {
    ...ProfessorNoteHeader_teacher
    ...ProfessorNoteFooter_teacher
  }

  fragment ProfessorNoteEditor_teacher on Teacher {
    id
  }

  fragment ProfessorNoteHeader_teacher on Teacher {
    lastName
  }

  fragment ProfessorNoteFooter_teacher on Teacher {
    legacyId
    isProfCurrentUser
  }
  """

  id = rmp_search(name)

  if id == "NA":
    return []

  variablesProbe = {"count":0,"id": id}
  url = "https://www.ratemyprofessors.com/graphql"
  basic = requests.auth.HTTPBasicAuth('test', 'test')
  resProbe = requests.post(url, json={"query": query, "variables": variablesProbe}, auth=basic)
  data = json.loads(resProbe.text)

  profRatingsCount = data['data']['node']['numRatings']

  variables = {"count":profRatingsCount,"id": id}
  res = requests.post(url, json={"query": query, "variables": variables}, auth=basic)
  ratings = json.loads(res.text)['data']['node']['ratings']['edges']

  return ratings

print(rmp_get_ratings("Clyde Kruskal"))

[{'cursor': 'YXJyYXljb25uZWN0aW9uOjA=', 'node': {'__typename': 'Rating', 'adminReviewedAt': None, 'attendanceMandatory': 'non mandatory', 'clarityRating': 5, 'class': 'CMSC751', 'comment': 'Best professor I ever had, knew what he was doing and seemed like he was very passionate. The class although difficult, was very well taught and made me more passionate about the subject.', 'createdByUser': False, 'date': '2022-12-02 17:49:57 +0000 UTC', 'difficultyRating': 1, 'flagStatus': 'UNFLAGGED', 'grade': 'A+', 'helpfulRating': 5, 'id': 'UmF0aW5nLTM2OTQ4MjAx', 'isForCredit': True, 'isForOnlineClass': False, 'legacyId': 36948201, 'ratingTags': 'Inspirational--Caring--Respected', 'teacherNote': None, 'textbookUse': -1, 'thumbs': [], 'thumbsDownTotal': 0, 'thumbsUpTotal': 0, 'wouldTakeAgain': 1}}, {'cursor': 'YXJyYXljb25uZWN0aW9uOjE=', 'node': {'__typename': 'Rating', 'adminReviewedAt': '2022-02-06 20:28:15 +0000 UTC', 'attendanceMandatory': 'non mandatory', 'clarityRating': 1, 'class': 'CMSC351

In [11]:
# Drop professors w/o any ratings / reviews
pt_df = pt_df.dropna(subset=["average_rating", "reviews"])

# Join names to help name search in salaries
for i, row in salaries_df.iterrows():
    parts = row["employee"].split(", ")
    # assert len(parts) == 2

    salaries_df.at[i, "name"] = " ".join(parts[::-1])

# Join names to help name search in rmp
def combineNames(row):
    if pd.isnull(row["tMiddlename"]):
        return row["tFname"] + " " + row["tLname"]
    return row["tFname"] + " " + row["tMiddlename"] + " " + row["tLname"]

rmp_df["fullname"] = rmp_df.apply(lambda row: combineNames(row), axis=1)


In [38]:
pt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2983 entries, 6 to 11693
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   courses         2983 non-null   object 
 1   average_rating  2983 non-null   float64
 2   type            2983 non-null   object 
 3   reviews         2983 non-null   object 
 4   name            2983 non-null   object 
 5   slug            2983 non-null   object 
dtypes: float64(1), object(5)
memory usage: 163.1+ KB


In [39]:
salaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107395 entries, 0 to 107394
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   year        107395 non-null  int64 
 1   employee    107395 non-null  object
 2   department  107395 non-null  object
 3   division    107395 non-null  object
 4   title       107395 non-null  object
 5   salary      107395 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.9+ MB


In [40]:
pt_df["type"].value_counts()

professor    2853
ta            130
Name: type, dtype: int64

In [41]:
pt_df["average_rating"].describe()

count    2983.000000
mean        3.893974
std         1.194694
min         1.000000
25%         3.000000
50%         4.200000
75%         5.000000
max         5.000000
Name: average_rating, dtype: float64

In [42]:
assert len(pt_df) == len(pt_df.drop_duplicates(subset=["slug"]))

In [43]:
salaries_df["employee"] = salaries_df["employee"].str.replace("\n", " ")

In [45]:
pt_names = pt_df["name"].unique()

for name in salaries_df["name"].unique()[:10]:
    match = process.extract(name, pt_names)
    print(f"{name} -> {match}")


Eyad H Abed -> [('Eyad Abed', 95), ('Zeyad Emam', 57), ('Cyrah Ward', 57), ('Behtash Babadi', 56), ('Adi Mahalel', 55)]
Pamela A. Abshire -> [('Pamela Abshire', 95), ('Pamela Orel', 86), ('A Anthony', 86), ('A Sharma', 86), ('Amde Amde', 64)]
Leigh R Abts -> [('R Apter', 86), ('Leigh Leslie', 58), ('Raley Roberts', 56), ('Leigha McReynolds', 55), ('Garth Libhart', 53)]
Douglas J Adams -> [('Douglas Hamilton (Astronomy)', 86), ('Douglas Hamilton (Physics)', 86), ('Douglas Oard', 74), ('Douglas Kern', 70), ('Douglas Julin', 66)]
Vincent G. Adams -> [('Vincent Lee', 74), ('Daniel Vincent', 66), ('Vincent Nguyen', 64), ('Brienne Adams', 62), ('Dawn Gavin', 60)]
Kofi Frimpong Addo -> [('Dong Dong', 64), ('Yang Tao', 56), ('David Simon', 55), ('Don DeVoe', 53), ('John Kim', 53)]
Raymond A. Adomaitis -> [('Raymond Adomaitis', 95), ('Raymond Weil', 86), ('A Kruglanski', 86), ('A Anthony', 86), ('A.U. Shankar', 86)]
Mohamed Sherif Aggour -> [('Mohamed Aggour', 86), ('Amy Fisher', 63), ('Mohamad

In [54]:
salary_names = salaries_df["name"].unique()

for name in pt_df["name"].unique()[:10]:
    # match = process.extract(name, salary_names)
    match = process.extractOne(name, salary_names, score_cutoff=90)
    print(f"{name} -> {match}")

    if match:
        salaries = salaries_df[salaries_df["name"] == match[0]]
        display(salaries)



Abhijit Dasgupta -> ('Abhijit Dasgupta', 100)


,year,employee,department,division,title,salary,name
195,2013,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Prof,"$167,138.22",Abhijit Dasgupta
12965,2014,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Prof,"$183,580.92",Abhijit Dasgupta
25384,2015,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Prof,"$190,895.40",Abhijit Dasgupta
35612,2016,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Prof,"$190,895.40",Abhijit Dasgupta
45733,2017,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Prof,"$198,038.26",Abhijit Dasgupta
56058,2018,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Prof,"$198,038.26",Abhijit Dasgupta
66508,2019,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Prof,"$201,999.03",Abhijit Dasgupta
77096,2020,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Professor,"$211,190.29",Abhijit Dasgupta
87770,2021,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark School of Engineering,Prof & Assoc Chair,"$215,414.09",Abhijit Dasgupta
98038,2022,"Dasgupta, Abhijit",ENGR-Mechanical Engineering,A. James Clark\nSchool of\nEngineering,Prof & Assoc Chair,"$227,086.06",Abhijit Dasgupta


Abigail McEwen -> ('Abigail McEwen', 100)


,year,employee,department,division,title,salary,name
2522,2013,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Asst Prof,"$64,260.00",Abigail McEwen
14073,2014,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Asst Prof,"$70,608.20",Abigail McEwen
26286,2015,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Asst Prof,"$73,910.42",Abigail McEwen
36480,2016,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Asst Prof,"$73,910.42",Abigail McEwen
46590,2017,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Assoc Prof,"$82,910.42",Abigail McEwen
56936,2018,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Assoc Prof,"$82,910.42",Abigail McEwen
67392,2019,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Assoc Prof,"$84,568.63",Abigail McEwen
78014,2020,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Associate Professor,"$93,721.62",Abigail McEwen
88605,2021,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts & Humanities,Associate Professor,"$95,596.05",Abigail McEwen
98847,2022,"McEwen, Abigail",ARHU-Art History & Archaeology,College of Arts\n& Humanities,Associate\nProfessor,"$98,965.81",Abigail McEwen


Abolhassan Jawahery -> ('Abolhassan Jawahery', 100)


...

,year,employee,department,division,title,salary,name
14673,2014,"Mahalel, Adi",ARHU-Meyerhoff Program & Center for Jewish Stu...,College of Arts & Humanities,Lecturer,"$56,650.00",Adi Mahalel
26761,2015,"Mahalel, Adi",ARHU-Meyerhoff Program & Center for Jewish Stu...,College of Arts & Humanities,Visit Assoc Prof,"$61,200.00",Adi Mahalel
36936,2016,"Mahalel, Adi",ARHU-Meyerhoff Program & Center for Jewish Stu...,College of Arts & Humanities,Visit Assoc Prof,"$61,200.00",Adi Mahalel
47074,2017,"Mahalel, Adi",ARHU-Meyerhoff Program & Center for Jewish Stu...,College of Arts & Humanities,Adjunct Asst Prof,"$61,984.63",Adi Mahalel
57409,2018,"Mahalel, Adi",ARHU-Meyerhoff Program & Center for Jewish Stu...,College of Arts & Humanities,Adjunct Asst Prof,"$61,984.63",Adi Mahalel
67888,2019,"Mahalel, Adi",ARHU-Meyerhoff Program & Center for Jewish Stu...,College of Arts & Humanities,Adjunct Asst Prof,"$63,224.32",Adi Mahalel
78518,2020,"Mahalel, Adi",ARHU-Meyerhoff Program & Center for Jewish Stu...,College of Arts & Humanities,Adjunct Asst Professor,"$66,101.12",Adi Mahalel
89061,2021,"Mahalel, Adi",ARHU-Meyerhoff Program & Center for Jewish Stu...,College of Arts & Humanities,Adjunct Asst Professor,"$67,423.14",Adi Mahalel
99297,2022,"Mahalel, Adi",ARHU-Meyerhoff Program &\nCenter for Jewish St...,College of Arts\n& Humanities,Adjunct Asst\nProfessor,"$70,140.37",Adi Mahalel


Adil Hassam -> ('Adil Hassam', 100)


,year,employee,department,division,title,salary,name
101413,2022,"Hassam, Adil",CMNS-Inst for Research in\nElectronics & Appli...,"College of\nComputer, Math\n& Natural\nSciences",Professor,"$44,731.34",Adil Hassam
101832,2022,"Hassam, Adil",CMNS-Physics,"College of\nComputer, Math\n& Natural\nSciences",Professor,"$67,096.99",Adil Hassam


Adriane Fang -> ('Adriane Fang', 100)


,year,employee,department,division,title,salary,name
2145,2013,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Asst Art-In-Res,"$51,000.00",Adriane Fang
15098,2014,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Asst Prof,"$61,800.00",Adriane Fang
27062,2015,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Asst Prof,"$64,897.81",Adriane Fang
37249,2016,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Asst Prof,"$64,897.81",Adriane Fang
47425,2017,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Asst Prof,"$70,000.00",Adriane Fang
57742,2018,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Asst Prof,"$70,000.00",Adriane Fang
68216,2019,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Asst Prof,"$71,400.00",Adriane Fang
78838,2020,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Assistant Professor,"$74,648.81",Adriane Fang
89423,2021,"Fang, Adriane","ARHU-School of Theatre, Dance & Performance St...",College of Arts & Humanities,Associate Professor,"$86,341.79",Adriane Fang
99621,2022,"Fang, Adriane","ARHU-School of Theatre, Dance\n& Performance S...",College of Arts\n& Humanities,Associate\nProfessor,"$89,385.34",Adriane Fang
